In [6]:
import torch
from paddleocr import PaddleOCR
import cv2
import numpy as np
import os
from PIL import Image
from docx import Document
import shutil
# need to run only once to download and load model into memory
ocr = PaddleOCR(use_angle_cls=True, lang="ch")

[2023/10/24 10:04:50] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\FWJ813/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\FWJ813/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_le

# 文件提取+预处理

In [10]:
class file_OCR:
    def __init__(self):
        self.file_num = []
    # 遍历文件夹中所有图片文件

    def recursive_listdir(self, path):

        files = os.listdir(path)
        for file in files:
            file_path = os.path.join(path, file)
            if os.path.isfile(file_path) and (file_path.lower().endswith(".jpg") or file_path.lower().endswith(".gif") or file_path.lower().endswith(".tif")):
                file_path = file_path.replace('\\', '/')
                print(file, file_path)
                self.file_num.append(file_path)

            elif os.path.isdir(file_path):
                self.recursive_listdir(file_path)

    # 测试文件转写是否可用
    def test_ocr(self, img_path):
        result = ocr.ocr(img_path, cls=True)
        for idx in range(len(result)):
            res = result[idx]
            for line in res:
                print(line[1][0])

    def extractFile(file):
        # 原始文件路径
        original_file = file

        # 提取原始文件名和扩展名
        filename, extension = os.path.splitext(os.path.basename(original_file))

        # 去除文件名中的中文字符
        filename = ''.join(x for x in filename if ord(x) < 128)

        # 构建新文件名
        new_filename = filename + '_processed' + extension

        # 复制文件到当前目录
        shutil.copy2(original_file, new_filename)

        return new_filename

    def needCut(file):
        # 打开图像
        image = Image.open(file)
        # 获取图像的长和宽
        width, length = image.size[0], image.size[1]
        if length < width:
            return True
        else:
            return False

    # 处理一个文件并生成docx
    def procPic(self, file, id):
        # 横版图片需要切分
        if self.needCut(file) == True:
            self.extractFile(file)
            split_file = self.cutPic(file)
            doc = Document()
            paragraph = doc.add_paragraph()
            for essay in split_file:
                result = ocr.ocr(essay, cls=True)
                for idx in range(len(result)):
                    res = result[idx]
                    if res:
                        for line in res:
                            if(line[1][1] > 0.8):
                                paragraph.add_run(line[1][0])
        # 竖版图片不需切分
        else:
            doc = Document()
            result = ocr.ocr(file, cls=True)
            paragraph = doc.add_paragraph()
            for idx in range(len(result)):
                res = result[idx]
                for line in res:
                    if(line[1][1] > 0.8):
                        paragraph.add_run(line[1][0])

        new_path = file.rsplit('.', 1)[0] + id + '.docx'
        doc.save(new_path)

    def procAllFile(self):
        num = 1
        for file in self.file_num:
            procPic(file, 'id{}'.format(num))
            num += 1


In [16]:
proc = file_OCR()
proc.recursive_listdir('./')
proc.procAllFile()


23320102710707.jpg ./1jiangsu/30.5-40/23320102710707.jpg
23320102810009.jpg ./1jiangsu/30.5-40/23320102810009.jpg
23320102850917.jpg ./1jiangsu/30.5-40/23320102850917.jpg
23320105850059.jpg ./1jiangsu/30.5-40/23320105850059.jpg
23320106710057.jpg ./1jiangsu/30.5-40/23320106710057.jpg
23320106710984.jpg ./1jiangsu/30.5-40/23320106710984.jpg
23320106711582.jpg ./1jiangsu/30.5-40/23320106711582.jpg
23320106810004.jpg ./1jiangsu/30.5-40/23320106810004.jpg
23320111840083.jpg ./1jiangsu/30.5-40/23320111840083.jpg
23320111860078.jpg ./1jiangsu/30.5-40/23320111860078.jpg
23320112810223.jpg ./1jiangsu/30.5-40/23320112810223.jpg
23320114850368.jpg ./1jiangsu/30.5-40/23320114850368.jpg
23320116850618.jpg ./1jiangsu/30.5-40/23320116850618.jpg
23320117810293.jpg ./1jiangsu/30.5-40/23320117810293.jpg
23320118740055.jpg ./1jiangsu/30.5-40/23320118740055.jpg
23320118810052.jpg ./1jiangsu/30.5-40/23320118810052.jpg
23320118860092.jpg ./1jiangsu/30.5-40/23320118860092.jpg
23320201711249.jpg ./1jiangsu/3

In [15]:
len(proc.file_num)

450